In [471]:
import numpy as np
import sys
sys.path.insert(0, '/Library/Application Support/MWorks/Scripting/Python')
from mworks.data import MWKFile
import pandas as pd

In [486]:
def unpack(filename):
    '''
    
    :param filename: Path to mwk2 file 
    :return: 
        data_scalar: Dataframe (n_trials x n_scalar_codenames) containing scalar data
        data_analog: Nothing (yet)
    '''
    
    def get_sync_times(f):
        '''
        Returns start and end time of each trial in event stream
        :param f: MWKFile object
        :return: 
            start_sync_t: array of trial start times
            end_sync_t: array of trial end times         
        '''
        sync_events = f.get_events(codes=['sync'])
        sync_events = np.asarray([[e.data, e.time] for e in sync_events])        
        end_sync_t = sync_events[np.where(sync_events[:, 0] == 6)[0], 1]
        start_sync_t = sync_events[np.where(sync_events[:, 0] == 0)[0], 1]
        
        
        start_sync_t = start_sync_t[:len(end_sync_t)]
        
        return start_sync_t, end_sync_t
    
    # Calculate spatial, symbolic distance, wrap, etc
    def calc_scalar_data(df):
        '''
        Calculates spatial, symbolic distance, wrap, cycle, option/selection distance for trials in df
        :param df: Dataframe containing scalar data from mwk2 stream. Modified by method        
        '''
        num_stims = 8 
        start_stim_index = df['start_stim_index']
        end_stim_index = df['end_stim_index']        
        stim_drift_direction = df['stim_drift_direction']
        stim_dist_cum = np.stack([np.asarray(a[0]) for a in df['stim_dist_cum']])        
        selected = df['selected']
        
        df['cycle'] = start_stim_index == end_stim_index
        df['wrap'] =  (stim_drift_direction * (end_stim_index - start_stim_index)) > 0
        df['symb_dist'] = stim_drift_direction * (
                (1 - df['cycle']) * (df['wrap'] * num_stims - np.abs(start_stim_index - end_stim_index)) \
                + df['cycle'] * num_stims)
        df['spat_dist'] = stim_drift_direction * (
                 (1 - df['cycle']) * (df['wrap'] * stim_dist_cum[:, -1] - np.abs(start_stim_index - end_stim_index)) \
                 + df['cycle'] * stim_dist_cum[:, -1])
        df['option_dist'] = abs(selected - end_stim_index)
        
        
    def get_scalar_data(f, codenames_scalar, start_sync_t, end_sync_t):
        N = len(start_sync_t) 
        data_scalar = {}
        for c in codenames_scalar:
            data_scalar[c] = []
        for trial in range(N):            
            start_t, end_t = np.long(start_sync_t[trial]), np.long(end_sync_t[trial])
            trial_events = f.get_events(codes = codenames_scalar, time_range=[start_t, end_t])
            trial_events = np.asarray([[e.code, e.time, e.data] for e in trial_events])              
            for ci, c in enumerate(codenames_scalar):                
                code = f.reverse_codec[c]
                code_events = trial_events[np.where(trial_events[:, 0] == code)[0], 2]
                if len(code_events) > 0:    
                    data_scalar[c].append(code_events)
                else:
                    data_scalar[c].append(None)                         
        data_scalar = pd.DataFrame(data_scalar)
        calc_scalar_data(data_scalar)
        return data_scalar
    
    def get_analog_data(f, codenames_analog, start_sync_t, end_sync_t):
        N = len(start_sync_t)
        data_analog = {}
        # for c in codenames_analog:
            
    fpath = '/Users/aidapiccato/Documents/MWorks/Data'
    f =  MWKFile('%s/%s' % (fpath, filename))
    f.open()
    all_events = f.get_events()
    codec = f.codec
    codenames_scalar = ['start_stim_index', 'end_stim_index', 'pair_index', 'options_bin', 'options_pos',
                        'ignore', 'success', 'failure', 'alpha_m', 'stim_dist_cum', 'stim_drift_direction', 'selected']
    codenames_analog = {'eye_h', 'eye_v', 'pointer_x', 'pointer_y'}
    start_sync_t, end_sync_t = get_sync_times(f)    
    print('Collecting %d trials of data' % len(start_sync_t))
    trial_dur_t = (end_sync_t - start_sync_t) / 10**6
    print('Average trial duration: %f seconds' % np.mean(trial_dur_t))
    print('Max trial duration: %f seconds' % np.amax(trial_dur_t))
    print('Min trial duration: %f seconds' % np.amin(trial_dur_t))    
    print(start_sync_t)
    data_scalar = get_scalar_data(f, codenames_scalar, start_sync_t, end_sync_t)
    data_analog = get_analog_data(f, codenames_analog, start_sync_t, end_sync_t)
    return data_scalar, data_analog
    

In [490]:
data_scalar, data_analog = unpack('aidapiccato-mental-nav-20191024-172009.mwk2')

Average trial duration: 11.321598 seconds
Max trial duration: 15.169998 seconds
Min trial duration: 5.744559 seconds
[24431437951 24440183793 24461701699 24473858945 24493385251 24507631058]


In [491]:
data_scalar['alpha_m']

0           [0]
1           [0]
2           [0]
3    [0.1, 0.1]
4         [0.1]
5         [0.1]
Name: alpha_m, dtype: object

In [492]:
f =  MWKFile('%s/%s' % (fpath, 'aidapiccato-mental-nav-20191024-172009.mwk2'))
f.open()
events = f.get_events()

sync_events = f.get_events(codes = ['sync'])
for e in sync_events:
    if e.data in [0, 6]:
        print(e.data, e.time)
sync_events = np.asarray([[e.data, e.time] for e in sync_events])        
end_sync_t = sync_events[np.where(sync_events[:, 0] == 6)[0], 1]
start_sync_t = sync_events[np.where(sync_events[0:, 0] == 0)[0], 1]

f.close()


0 24431437951
6 24437182510
0 24440183793
6 24455353791
0 24461701699
6 24470857590
0 24473858945
6 24486564387
0 24493385251
6 24504629356
0 24507631058
6 24521540649


In [480]:
8 * 8 * 2



128